In [9]:
import ast

In [23]:
ast.dump(ast.parse('if x >= 0:\n    x = 0'))

"Module(body=[If(test=Compare(left=Name(id='x', ctx=Load()), ops=[GtE()], comparators=[Num(n=0)]), body=[Assign(targets=[Name(id='x', ctx=Store())], value=Num(n=0))], orelse=[])])"

In [19]:
dir(ast.parse('test_var = test_list[0]').body[0])

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_attributes',
 '_fields',
 'col_offset',
 'lineno',
 'targets',
 'value']

## Импорты

In [6]:
%matplotlib inline
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import os
from tqdm import tqdm_notebook
import pickle
import numpy as np
from IPython.display import clear_output

# Предобработка

In [21]:
base_path = '/Users/dentarasov/Yandex.Disk.localized/current/vkr'
code_batched = base_path + '/after_preprocess/after_preprocess.part0'
params = {
    'batch_size': 64
}


In [ ]:
def build_vocab():
    # load pre-computed vocab
    with open(base_path + '/mapping.map', 'rb') as f:
        word_to_id = pickle.load(f)
    id_to_word = dict([(v, k) for (k, v) in vocab.items()])
    return word_to_id, id_to_word

In [4]:
def get_code_data(path):
    for _, _, files in os.walk(path):
        data = np.empty((0, 100))
        for file in tqdm_notebook(files):
#             print(path + file)
            with open(path + file, 'rb') as f:
                array = pickle.load(f)
                for i in tqdm_notebook(range(len(array)), leave=False):
                    data = np.concatenate((data, array[i].inputs), axis=0)
#                     print(array[i].inputs.shape)
#                     print(array[i].inputs[0].shape)
#                     break
#             break
#         break
    return data

In [5]:
data = get_code_data(base_path + '/after_preprocess/')

In [7]:
data.shape

(695687, 100)

In [14]:
# with open('./after_preprocess/all_inputs', 'wb') as f:
#     pickle.dump(data, f)

In [22]:
data_path = base_path + '/after_preprocess/all_inputs'

In [24]:
class CodeDataset(Dataset):
    def __init__(self, path):
        with open(path, 'rb') as f:
            self.data = pickle.load(f)
    
    def __len__(self):
        return self.data.shape[0]
    
    def __getitem__(self, index):
        return self.data[index]

In [25]:
train_loader = DataLoader(
    CodeDataset(data_path),
    batch_size=params['batch_size'],      
    shuffle=True,
    num_workers=10
)

# Инициализация модели

# Обучение

In [ ]:
# def train_epoch(model, optimizer, lr):
#     loss_log = []
#     model.train()
    
#     for batch_num, (x, y) in zip(trange(len(train_loader)), train_loader):
#         optimizer.zero_grad()
# #         x, y = x.to(device), y.to(device)
#         params['batch_size'] = x.shape[0]
#         p = torch.Tensor(generate_mask(params)).to(device)
#         hidden = torch.zeros(2, 2, params['batch_size'], params["gen_rnn_size"], device=device)
#         gen_output, logit_out = model.forward(x, y, p, hidden)
#         loss_value = 0
#         costs = 0
#         loss = nn.CrossEntropyLoss()
#         for i in range(params['sequence_length']):
#             indices = torch.nonzero(p[:, i]).squeeze()
#             if indices.dim() == 0:
#                 loss_value += loss(logit_out.float()[:, i][indices].view(1, -1), y[:, i][indices].view(1))
#             else:
#                 loss_value += loss(logit_out.float()[:, i][indices], y[:, i][indices])
#         loss_value.backward()
#         torch.nn.utils.clip_grad_norm(model.parameters(), 0.5)
#         for p in model.parameters():
#             p.data.add_(-lr, p.grad.data)
#         optimizer.step()
        
#         loss_value = loss_value.item()
#         loss_log.append(loss_value / params['sequence_length'])
#     perp = np.exp(np.mean(loss_log))
#     return loss_log, perp

# def test(model, test_batches):
#     loss_log = []
#     model.eval()
#     for batch_num, (x, y) in zip(trange(len(train_loader)), val_loader):        
#         hidden = model.init_hidden(batch.shape[0])
#         loss = 0
#         gen_output = model.forward(x, y)
#         loss = F.cross_entropy(gen_output.float(), y.float())
#         loss = loss.item()
#         loss_log.append(loss / nums.shape[0])
#     return loss_log

# def plot_history(train_history, title='loss'):
#     plt.figure()
#     plt.title('{}'.format(title))
#     plt.plot(train_history, label='train', zorder=1)
#     plt.xlabel('train steps')
#     plt.legend(loc='best')
#     plt.grid()
#     plt.show()
    
# def train(model, opt, n_epochs):
#     train_log = []
#     val_log = []
#     perp_log = []
#     lr = 0.05
#     lr_decay_base = 1 / 1.15
#     m_flat_lr = 20.0
#     for epoch in range(n_epochs):
#         lr_decay = lr_decay_base ** max(epoch - m_flat_lr, 0)
#         lr = lr * lr_decay
#         train_loss, perp = train_epoch(model, opt, lr)
#         train_log.extend(train_loss)
#         perp_log.append(perp)
#         if (epoch + 1) % 10 == 0:
#             torch.save({
#             'epoch': epoch,
#             'model_state_dict': model.state_dict(),
#             'optimizer_state_dict': opt.state_dict(),
#             'loss': train_log[-1]}, '/home/.../model_checkpoints/..._model.checkpoint_{}.pt'.format(epoch))
#         clear_output()
#         print("Epoch:{}".format(epoch))
#         plot_history(train_log)
#         plot_history(perp_log)
#     torch.save({
#             'epoch': n_epochs,
#             'model_state_dict': model.state_dict(),
#             'optimizer_state_dict': opt.state_dict(),
#             'loss': train_log[-1]}, '/home/.../model_checkpoints..._model.checkpoint_{}.pt'.format(n_epochs))
#     np.save("/home/.../model_checkpoints..._logs.npy", np.array(train_log))
#     np.save("/home/.../model_checkpoints..._logs_perp.npy", np.array(perp_log))
    